# 🔧 One‑Click YOLOv5 Training Notebook

This Colab notebook lets **anyone** train a custom YOLOv5 object‑detection model with just a few form fields—no coding required.

**How it works**

1. Fill in the form under **“📋 Configure your training”** with your Roboflow dataset info and training settings.  
2. Run each cell in order (⇧ **Shift+Enter** or click **Run all**).  
3. The notebook will download your dataset, train the model, export an *INT8‑quantized* TensorFlow‑Lite file, and save both the model and full training logs to your Google Drive.

> **Tip:** If something goes wrong, just edit the form parameters and rerun the cell that failed (and any below it).

In [ ]:
#@title ⬇️ Install YOLOv5 (first run takes ~1 min)
%cd /content
!git clone -q https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -q -r requirements.txt


In [ ]:
#@title ⬇️ Install Roboflow helper (few seconds)
!pip install roboflow
#!pip install utils

In [ ]:
#@title 📋 Configure your training  — fill in & run every time you want a new model
from datetime import datetime

# --- Roboflow creds
api_key      = "dcODmr5cR6PQjYCXpQHe"          #@param {type:"string"}
workspace    = "steptracking-objectdetection-2"   #@param {type:"string"}
project_slug = "dy08-p7-st-all-eh-na0k-k"         #@param {type:"string"}
version_num  = 3                                #@param {type:"integer"}

# --- Hyper-parameters
model_size = "yolov5n.pt"       #@param ["yolov5n.pt","yolov5s.pt","yolov5m.pt","yolov5l.pt","yolov5x.pt"]
img_size   = 256                #@param {type:"integer"}
batch_size = 64                 #@param {type:"integer"}
epochs     = 80                 #@param {type:"integer"}

# --- Where to store outputs on Drive
drive_root = "/content/drive/My Drive"          #@param {type:"string"}

# ==========  auto-generate a unique run name  ==========
stamp    = datetime.now().strftime("%Y%m%d-%H%M")
RUN_NAME = f"{project_slug}_{stamp}"
RUN_DIR  = f"runs/train/{RUN_NAME}"             # YOLOv5 will create this
print(f"🏷️  This run will be saved to → {RUN_DIR}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ⬇️ Download dataset from Roboflow
from roboflow import Roboflow, util
rf        = Roboflow(api_key="dcODmr5cR6PQjYCXpQHe")
dataset   = rf.workspace(workspace).project(project_slug).version(version_num).download("yolov5")
print("✅ Dataset:", dataset.location)

In [ ]:
#@title 📈 Dashboard
%pip install -q --no-warn-conflicts tensorboard
%load_ext tensorboard

import subprocess, pathlib

TB_LOGDIR = str(pathlib.Path.cwd() / "runs/train")   # parent dir (TensorBoard will watch RUN_DIR as it appears)
subprocess.run(["pkill", "-f", "tensorboard"], check=False)
%tensorboard --logdir $TB_LOGDIR --port 0 --reload_interval 30


In [ ]:
#@title 🚀 Train YOLOv5 (grab a coffee ☕ – this can take 5‑30 min)
# 🔧 Auto-fix data.yaml if val / test splits are missing
import os, yaml, pathlib
data_yaml = f"{dataset.location}/data.yaml"  # `dataset` is defined in the previous cell
with open(data_yaml) as f:
    data = yaml.safe_load(f)

def _need_fix(key):
    path = pathlib.Path(dataset.location) / pathlib.Path(data[key])
    return not path.exists()

changed = False
for key in ("val", "test"):
    if _need_fix(key):
        print(f"⚠️  {key} split not found – pointing it to the train split.")
        data[key] = data["train"]
        changed = True

if changed:
    with open(data_yaml, "w") as f:
        yaml.safe_dump(data, f)
    print("✅ data.yaml patched. Training will proceed without validation metrics.")
else:
    print("✅ All splits present – no patching needed.")

!python train.py --img {img_size} --batch {batch_size} \
      --epochs {epochs} --data {dataset.location}/data.yaml \
      --weights {model_size} --project runs/train --name {RUN_NAME} --exist-ok


In [ ]:
#@title 📦 Export best model to TensorFlow‑Lite (INT8) 💾 Copy exported model file to Drive & 📂 Backup full training logs to Drive (optional)
# 📦 Export INT8 TFLite
BEST_PT   = f"{RUN_DIR}/weights/best.pt"
!python export.py --weights {BEST_PT} --include tflite --imgsz {img_size}

# 🔗 Mount Drive (once per session)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 💾 Copy files to Drive
import pathlib, shutil

BEST_FP16 = f"{RUN_DIR}/weights/best-fp16.tflite"
BEST_INT8 = f"{RUN_DIR}/weights/best-int8.tflite"
drive_exp = pathlib.Path(drive_root) / "TrainingResults" / RUN_NAME
drive_exp.mkdir(parents=True, exist_ok=True)

for src in (BEST_PT, BEST_FP16, BEST_INT8):
    if pathlib.Path(src).exists():
        shutil.copy(src, drive_exp)
        print(f"✅ Copied {pathlib.Path(src).name} → {drive_exp}")
    else:
        print(f"❌ Missing {src}")

# (optional) backup full TensorBoard logs
shutil.copytree(RUN_DIR, drive_exp / "run_logs", dirs_exist_ok=True)
print("📂 Full run backed up.")
